In [21]:
import os
import json
import unidecode
from collections import Counter

In [22]:
# Đường dẫn tới dữ liệu
DATA_PATH = "../data/processed/vqa_data.json"
IMAGE_DIR = "../data/raw/"

In [23]:
# Load dữ liệu JSON
try:
    with open(DATA_PATH, "r", encoding="utf-8") as f:
        vqa_data = json.load(f)
    print(f"Dữ liệu đã tải thành công! Tổng số ảnh: {len(vqa_data)}\n")
except Exception as e:
    print(f"Lỗi khi tải dữ liệu: {e}")

Dữ liệu đã tải thành công! Tổng số ảnh: 8579



In [24]:
# Kiểm tra ảnh có tồn tại không
missing_images = []
for item in vqa_data:
    image_path = os.path.join(IMAGE_DIR, item["image_id"])
    if not os.path.exists(image_path):
        missing_images.append(image_path)

if missing_images:
    print(f"Có {len(missing_images)} ảnh không tồn tại:")
    for img in missing_images[:5]:  # Chỉ hiển thị 5 ảnh đầu tiên nếu nhiều
        print(f"    - {img}")
    if len(missing_images) > 5:
        print(f"    ...và {len(missing_images) - 5} ảnh khác")
else:
    print("Tất cả ảnh đều tồn tại!\n")

Tất cả ảnh đều tồn tại!



In [25]:
# Kiểm tra câu hỏi và câu trả lời
error_questions = []
error_answers = []

for item in vqa_data:
    image_id = item["image_id"]
    questions = item.get("questions", [])

    if len(questions) != 4:  # Kiểm tra có đúng 4 câu hỏi không
        error_questions.append((image_id, f"Số câu hỏi: {len(questions)}"))

    for q in questions:
        if "question" not in q or "correct_answer" not in q:
            error_answers.append((image_id, "Thiếu câu hỏi hoặc đáp án"))

if error_questions:
    print(f"Có {len(error_questions)} ảnh có số câu hỏi không hợp lệ:")
    for img, err in error_questions[:5]:
        print(f"    - {img}: {err}")
else:
    print("Tất cả ảnh đều có đúng 4 câu hỏi!\n")

if error_answers:
    print(f"Có {len(error_answers)} câu trả lời không hợp lệ:")
    for img, err in error_answers[:5]:
        print(f"    - {img}: {err}")
else:
    print("Tất cả câu hỏi đều có đáp án hợp lệ!\n")

Tất cả ảnh đều có đúng 4 câu hỏi!

Tất cả câu hỏi đều có đáp án hợp lệ!



In [26]:
# Kiểm tra lỗi mã hóa
encoding_errors = []
for item in vqa_data:
    for q in item["questions"]:
        try:
            question_text = q["question"]
            answer_text = q["correct_answer"]
            if (
                unidecode.unidecode(question_text) == ""
                or unidecode.unidecode(answer_text) == ""
            ):
                encoding_errors.append((item["image_id"], question_text, answer_text))
        except Exception as e:
            encoding_errors.append((item["image_id"], f"Lỗi mã hóa: {e}"))

if encoding_errors:
    print(f"Có {len(encoding_errors)} câu hỏi/đáp án có thể bị lỗi mã hóa:")
    for img, q, a in encoding_errors[:5]:
        print(f"    - {img}: Câu hỏi: '{q}', Đáp án: '{a}'")
else:
    print("Không có lỗi mã hóa!\n")

Không có lỗi mã hóa!



In [27]:
# Thống kê bổ sung
question_types = Counter()
answer_dist = Counter()
unclear_count = 0

for item in vqa_data:
    for q in item["questions"]:
        question_text = q["question"].lower()
        answer = q["correct_answer"]

        # Phân loại câu hỏi
        if "đây là quả gì" in question_text or "tên gì" in question_text:
            question_types["Nhận diện"] += 1
        elif "bao nhiêu" in question_text or "số lượng" in question_text:
            question_types["Số lượng"] += 1
        elif "màu gì" in question_text or "màu sắc" in question_text:
            question_types["Màu sắc"] += 1
        elif "ở đâu" in question_text or "vị trí" in question_text:
            question_types["Vị trí"] += 1

        # Thống kê đáp án
        answer_dist[answer] += 1
        if answer == "không rõ":
            unclear_count += 1

In [28]:
# Hiển thị thống kê
print("=== Thống kê dữ liệu ===")
print(f"Tổng số câu hỏi: {sum(question_types.values())}")
print("Phân bố loại câu hỏi:")
for q_type, count in question_types.items():
    print(
        f"    - {q_type}: {count} ({count / sum(question_types.values()) * 100:.2f}%)"
    )

print("\nPhân bố đáp án (10 đáp án phổ biến nhất):")
for answer, count in answer_dist.most_common(10):
    print(f"    - '{answer}': {count} ({count / sum(answer_dist.values()) * 100:.2f}%)")

print(
    f"\nSố lượng đáp án 'không rõ': {unclear_count} ({unclear_count / sum(answer_dist.values()) * 100:.2f}%)\n"
)

# Kết luận
if not (missing_images or error_questions or error_answers or encoding_errors):
    print("Dữ liệu hợp lệ và sẵn sàng sử dụng!")
else:
    print("Dữ liệu có lỗi cần khắc phục!")

=== Thống kê dữ liệu ===
Tổng số câu hỏi: 33656
Phân bố loại câu hỏi:
    - Nhận diện: 8496 (25.24%)
    - Số lượng: 8557 (25.42%)
    - Màu sắc: 8564 (25.45%)
    - Vị trí: 8039 (23.89%)

Phân bố đáp án (10 đáp án phổ biến nhất):
    - 'không rõ': 7615 (22.19%)
    - 'cây': 2646 (7.71%)
    - 'đỏ': 2423 (7.06%)
    - 'xanh': 1945 (5.67%)
    - 'vàng': 1408 (4.10%)
    - 'nâu': 1327 (3.87%)
    - '2': 975 (2.84%)
    - 'bàn': 912 (2.66%)
    - '1': 862 (2.51%)
    - '3': 766 (2.23%)

Số lượng đáp án 'không rõ': 7615 (22.19%)

Dữ liệu hợp lệ và sẵn sàng sử dụng!
